# Xây dựng hệ thống đề xuất phim sử dụng thuật toán Collaborative filtering

In [17]:
import os
import numpy as np
from scipy import optimize
from scipy.io import loadmat
import utils

## Recommender Systems

Bài toán này, ta sẽ thực hiện thuật toán Collaborative filtering và áp dụng nó vào bộ dữ liệu xếp hạng phim  Bộ dữ liệu này bao gồm các xếp hạng theo thang điểm từ 1 đến 5. Bộ dữ liệu có $n_u = 943 $ người dùng và $n_m = 1682 $ phim.

Chúng ta sẽ đi xây dựng hàm cofiCostFunc để tính toán hàm mục tiêu Collaborative filtering và độ dốc. Sau khi thực hiện hàm chi phí và độ dốc, ta sẽ sử dụng scipy.optizes.minizing để tìm hiểu các parameter.

**1 Tập dữ liệu xếp hạng phim**

Cell sau sẽ tải tập dữ liệu ex8_movies.mat, cung cấp các biến Y và R. Ma trận Y (a num_movies $\times$ num_users matrix) lưu trữ các xếp hạng $ y ^ {(i, j)} $ (từ 1 đến 5 ). Ma trận R là ma trận nhị phân, trong đó $ R (i, j) = 1 $ nếu người dùng $ j $ đưa ra xếp hạng cho phim $ i $ và $ R (i, j) = 0 $ nếu không. Mục tiêu của Collaborative filtering là dự đoán xếp hạng phim cho các phim mà người dùng chưa xếp hạng, nghĩa là các mục có $ R (i, j) = 0 $. Điều này sẽ cho phép chúng ta đề xuất những bộ phim có xếp hạng dự đoán cao nhất cho người dùng.

Sau đây sẽ tính xếp hạng phim trung bình cho phim đầu tiên (Toy Story) và print xếp hạng trung bình của nó.
 

In [25]:
data = loadmat( 'ex8_movies.mat')
Y, R = data['Y'], data['R']

# Y là ma trận 1682x943, chứa xếp hạng (1-5) của 1682 phim trên 943 người dùng

# R là ma trận 1682x943, trong đó R (i, j) = 1 
# khi và chỉ khi người dùng j đưa ra xếp hạng cho phim i

# Từ ma trận, chúng ta có thể tính toán các thống kê như xếp hạng trung bình.
print('Xếp hạng trung bình cho phim 1 (Toy Story): %f / 5' %
      np.mean(Y[0, R[0, :]]))

Xếp hạng trung bình cho phim 1 (Toy Story): 4.520679 / 5


Ta cũng sẽ làm việc với các ma trận, X và Theta:
 
![](https://render.githubusercontent.com/render/math?math=%5Ctext%7BX%7D%20%3D%20%0A%5Cbegin%7Bbmatrix%7D%0A-%20%5Cleft%28x%5E%7B%281%29%7D%5Cright%29%5ET%20-%20%5C%5C%0A-%20%5Cleft%28x%5E%7B%282%29%7D%5Cright%29%5ET%20-%20%5C%5C%0A%5Cvdots%20%5C%5C%0A-%20%5Cleft%28x%5E%7B%28n_m%29%7D%5Cright%29%5ET%20-%20%5C%5C%0A%5Cend%7Bbmatrix%7D%2C%20%5Cquad%0A%5Ctext%7BTheta%7D%20%3D%20%0A%5Cbegin%7Bbmatrix%7D%0A-%20%5Cleft%28%5Ctheta%5E%7B%281%29%7D%5Cright%29%5ET%20-%20%5C%5C%0A-%20%5Cleft%28%5Ctheta%5E%7B%282%29%7D%5Cright%29%5ET%20-%20%5C%5C%0A%5Cvdots%20%5C%5C%0A-%20%5Cleft%28%5Ctheta%5E%7B%28n_u%29%7D%5Cright%29%5ET%20-%20%5C%5C%0A%5Cend%7Bbmatrix%7D.&mode=display)

$I^{th} $ hàng X tương ứng với vectơ đặc trưng $ x ^ {(i)} $ cho phim $ i ^ {th} $ và hàng $ j ^ {th} $ của Theta tương ứng với một vectơ tham số $ \theta ^ {(j)} $, cho người dùng $ j ^ {th} $. Cả $ x ^ {(i)} $ và $ \theta ^ {(j)} $ đều là các vectơ n chiều. Với mục đích của bài này, ta sẽ sử dụng $ n = 100 $ và do đó, $ x ^ {(i)} \in \mathbb {R} ^ {100} $ và $ \theta ^ {(j)} \in \mathbb {R} ^ {100} $. Tương ứng, X là ma trận $ n_m \times 100 $ và Theta là ma trận $ n_u \times 100 $.

**2 Thuật toán Collaborative filtering**

Hàm cofiCostFunc để tính toán chi phí và độ dốc cho thuật toán.

**2.1 Hàm chi phí với regularization**

Hàm chi phí cho lọc cộng tác với chính quy được đưa ra bởi

![](https://render.githubusercontent.com/render/math?math=J%28x%5E%7B%281%29%7D%2C%20%5Cdots%2C%20x%5E%7B%28n_m%29%7D%2C%20%5Ctheta%5E%7B%281%29%7D%2C%20%5Cdots%2C%20%5Ctheta%5E%7B%28n_u%29%7D%29%20%3D%20%5Cfrac%7B1%7D%7B2%7D%20%5Csum_%7B%28i%2Cj%29%3Ar%28i%2Cj%29%3D1%7D%20%5Cleft%28%20%5Cleft%28%20%5Ctheta%5E%7B%28j%29%7D%20%5Cright%29%5ET%20x%5E%7B%28i%29%7D%20-%20y%5E%7B%28i%2Cj%29%7D%20%5Cright%29%5E2%20%2B%20%5Cleft%28%20%5Cfrac%7B%5Clambda%7D%7B2%7D%20%5Csum_%7Bj%3D1%7D%5E%7Bn_u%7D%20%5Csum_%7Bk%3D1%7D%5E%7Bn%7D%20%5Cleft%28%20%5Ctheta_k%5E%7B%28j%29%7D%20%5Cright%29%5E2%20%20%5Cright%29%20%2B%20%5Cleft%28%20%5Cfrac%7B%5Clambda%7D%7B2%7D%20%5Csum_%7Bi%3D1%7D%5E%7Bn_m%7D%20%5Csum_%7Bk%3D1%7D%5En%20%5Cleft%28x_k%5E%7B%28i%29%7D%20%5Cright%29%5E2%20%5Cright%29&mode=display)

**2.2 Độ dốc với regularization**

Bây giờ ta đã thực hiện hàm chi phí regularization, thực hiện regularization cho gradient. 

![](https://render.githubusercontent.com/render/math?math=%5Cfrac%7B%5Cpartial%20J%7D%7B%5Cpartial%20x_k%5E%7B%28i%29%7D%7D%20%3D%20%5Csum_%7Bj%3Ar%28i%2Cj%29%3D1%7D%20%5Cleft%28%20%5Cleft%28%5Ctheta%5E%7B%28j%29%7D%5Cright%29%5ET%20x%5E%7B%28i%29%7D%20-%20y%5E%7B%28i%2Cj%29%7D%20%5Cright%29%20%5Ctheta_k%5E%7B%28j%29%7D%20%2B%20%5Clambda%20x_k%5E%7B%28i%29%7D%20%24%24%24%24%20%5Cfrac%7B%5Cpartial%20J%7D%7B%5Cpartial%20%5Ctheta_k%5E%7B%28j%29%7D%7D%20%3D%20%5Csum_%7Bi%3Ar%28i%2Cj%29%3D1%7D%20%5Cleft%28%20%5Cleft%28%5Ctheta%5E%7B%28j%29%7D%5Cright%29%5ET%20x%5E%7B%28i%29%7D-%20y%5E%7B%28i%2Cj%29%7D%20%5Cright%29%20x_k%5E%7B%28j%29%7D%20%2B%20%5Clambda%20%5Ctheta_k%5E%7B%28j%29%7D&mode=display)

Điều này có nghĩa là ta chỉ cần thêm $ \lambda x ^ {(i)} $ vào biến X_grad [i ,:] được mô tả trước đó và thêm $ \lambda \theta ^ {(j)} $ vào Theta_grad [j ,:] biến được mô tả trước đó.

In [19]:
def cofiCostFunc(params, Y, R, num_users, num_movies,
                      num_features, lambda_=0.0):
    
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)

    J = (1 / 2) * np.sum(np.square((X.dot(Theta.T) - Y) * R)) + (lambda_ / 2) * np.sum(np.square(X)) + \
                                                                (lambda_ / 2) * np.sum(np.square(Theta))
    
    for i in range(R.shape[0]):
        
        idx = np.where(R[i, :] == 1)[0]
        Theta_temp = Theta[idx, :]
        Y_temp = Y[i, idx]
        X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp) + lambda_ * X[i, :]
        
    for j in range(R.shape[1]):
        
        idx = np.where(R[:, j] == 1)[0]
        X_temp = X[idx, :]
        Y_temp = Y[idx, j]
        Theta_grad[j, :] = np.dot(np.dot(X_temp, Theta[j, :]) - Y_temp, X_temp) + lambda_ * Theta[j, :]
             
    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    
    return J, grad

**3 Đề xuất phim**

Sau khi ta hoàn thành việc thực hiện chức năng cofiCostFunc, bây giờ ta có thể bắt đầu đào tạo thuật toán của mình để đưa ra đề xuất phim khi muốn giới thiệu cho người dùng. Trong ô tiếp theo, ta có thể nhập tùy chọn phim của riêng mình, để sau  khi thuật toán chạy, ta có thể nhận các đề xuất phim của riêng mình. Danh sách tất cả các phim và idx của chúng trong tập dữ liệu movie_idx.txt.
 


In [20]:
# Trước khi chúng ta đào tạo mô hình, trước tiên chúng ta sẽ thêm 
# xếp hạng tương ứng với người dùng mới mà chúng ta vừa khảo sát.
n_m = 1682

#  Initialize my ratings
my_ratings = np.zeros(n_m)

# Kiểm tra tệp movie_idx.txt để biết id của từng phim trong tập dữ liệu của chúng tôi Ví dụ: 
# Toy Story (1995) có ID 1, vì vậy ta xếp hạng 4
my_ratings[0] = 4

# Hoặc giả sử không thích Silence of the Lambs(1991), ta có thể đặt
my_ratings[97] = 2

# Tôi đã chọn một vài bộ phim thích/không thích và xếp hạng chúng :
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63] = 5
my_ratings[65] = 3
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5

Sau khi xếp hạng bổ sung đã được thêm vào tập dữ liệu, tập lệnh sẽ tiến hành đào tạo mô hình lọc cộng tác. Điều này sẽ tìm hiểu các tham số X và Theta. Để dự đoán xếp hạng phim i cho người dùng j, bạn cần tính (θ (j)) T x (i). Phần tiếp theo sẽ tính toán xếp hạng cho tất cả các phim và người dùng và hiển thị các phim mà nó đề xuất theo xếp hạng đã được nhập trước đó . Lưu ý rằng ta có thể có được một bộ dự đoán khác nhau do các giá trị khởi tạo ngẫu nhiên khác nhau.

In [28]:
#  Bây giờ, bạn sẽ đào tạo mô hình lọc cộng tác trên bộ dữ liệu xếp hạng phim gồm 1682 phim và 943 người dùng
#  Load data
data = loadmat( 'ex8_movies.mat')
Y, R = data['Y'], data['R']
#  Y là ma trận 1682x943, chứa xếp hạng (1-5) trong số 1682 phim bởi 943 người dùng
#  R là ma trận 1682x943, trong đó R (i, j) = 1 khi và chỉ khi người dùng j đánh giá cao phim i

#  Thêm xếp hạng của chúng ta vào ma trận dữ liệu
Y = np.hstack([my_ratings[:, None], Y])
R = np.hstack([(my_ratings > 0)[:, None], R])

#  Normalize Ratings
Ynorm, Ymean = utils.normalizeRatings(Y, R)

num_movies, num_users = Y.shape
num_features = 10

X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Đặt các tùy chọn cho scipy.optimize.minimize
options = {'maxiter': 100}

# Set Regularization
lambda_ = 10
res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,
                                               num_movies, num_features, lambda_),
                        initial_parameters,
                        method='TNC',
                        jac=True,
                        options=options)
theta = res.x

# Unfold the returned theta back into U and W
X = theta[:num_movies*num_features].reshape(num_movies, num_features)
Theta = theta[num_movies*num_features:].reshape(num_users, num_features)

print('Đã hoàn thành việc học hệ thống đề xuất.')
#Sau khi đào tạo mô hình, bây giờ bạn có thể đưa ra các đề xuất bằng cách tính toán ma trận dự đoán.

Đã hoàn thành việc học hệ thống đề xuất.


In [29]:
p = np.dot(X, Theta.T)
my_predictions = p[:, 0] + Ymean

movieList = utils.loadMovieList()

ix = np.argsort(my_predictions)[::-1]

print('Các đề xuất hàng đầu cho bạn:')
print('----------------------------')
for i in range(10):
    j = ix[i]
    print('Dự đoán %.1f sao, cho phim %s' % (my_predictions[j], movieList[j]))

print('\nXếp hạng ban đầu được cung cấp:')
print('--------------------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Đã đánh giá %d sao, cho %s' % (my_ratings[i], movieList[i]))

Các đề xuất hàng đầu cho bạn:
----------------------------
Dự đoán 5.0 sao, cho phim Someone Else's America (1995)
Dự đoán 5.0 sao, cho phim Entertaining Angels: The Dorothy Day Story (1996)
Dự đoán 5.0 sao, cho phim Great Day in Harlem, A (1994)
Dự đoán 5.0 sao, cho phim Aiqing wansui (1994)
Dự đoán 5.0 sao, cho phim They Made Me a Criminal (1939)
Dự đoán 5.0 sao, cho phim Marlene Dietrich: Shadow and Light (1996)
Dự đoán 5.0 sao, cho phim Star Kid (1997)
Dự đoán 5.0 sao, cho phim Santa with Muscles (1996)
Dự đoán 5.0 sao, cho phim Prefontaine (1997)
Dự đoán 5.0 sao, cho phim Saint of Fort Washington, The (1993)

Xếp hạng ban đầu được cung cấp:
--------------------------
Đã đánh giá 4 sao, cho Toy Story (1995)
Đã đánh giá 3 sao, cho Twelve Monkeys (1995)
Đã đánh giá 5 sao, cho Usual Suspects, The (1995)
Đã đánh giá 4 sao, cho Outbreak (1995)
Đã đánh giá 5 sao, cho Shawshank Redemption, The (1994)
Đã đánh giá 3 sao, cho While You Were Sleeping (1995)
Đã đánh giá 5 sao, cho Forrest Gump